Analysing with Decision tree
Setup: put it in the same folder as "unifed_csv_without_duplicated_company.csv" and "ROI.csv" (generated by get_roi notebook) 

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split   
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier 
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from IPython.display import Image
import graphviz
from sklearn.model_selection import GridSearchCV
from statistics import mean

pd.set_option('display.max_columns', None)
# Optional
# import library for visuaiization
from sklearn import tree
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
#Read CSV
df=pd.read_csv("unified_csv.csv")
df.info(max_cols=1000)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4050 entries, 0 to 4049
Data columns (total 96 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   company_name                                 4050 non-null   object 
 1   status                                       4050 non-null   object 
 2   category_group_list                          4050 non-null   object 
 3   num_funding_rounds                           4050 non-null   float64
 4   total_funding_usd                            4050 non-null   float64
 5   founded_on                                   4050 non-null   float64
 6   employee_count                               4050 non-null   object 
 7   cat_commerce_shopping                        4050 non-null   float64
 8   cat_fin_services                             4050 non-null   float64
 9   cat_lending_invests                          4050 non-null   float64
 10  

In [3]:
#Preparing xlabel and ylabel
features= df[["company_name","num_funding_rounds", "fd_rd_latest_investment", "cat_lending_invests","cat_payments", "cat_commerce_shopping",  "first_fund_investor_count", "first_fund_post_money","num_exec","fd_rd_num_invested_by_top_100" ]]


ylabel=df[["average_momentum"]]



features

,company_name,num_funding_rounds,fd_rd_latest_investment,cat_lending_invests,cat_payments,cat_commerce_shopping,first_fund_investor_count,first_fund_post_money,num_exec,fd_rd_num_invested_by_top_100
0,All of Us,3.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,PayPlug,5.0,1.0,0.0,1.0,1.0,2.0,0.0,6.0,0.0
2,Zeitgold,5.0,3.0,0.0,0.0,0.0,2.0,0.0,11.0,2.0
3,AID:Tech,10.0,2.0,0.0,0.0,0.0,10.0,0.0,19.0,1.0
4,OwlTing,4.0,2.0,0.0,0.0,1.0,0.0,0.0,6.0,0.0
...,...,...,...,...,...,...,...,...,...,...
4045,Insurello,4.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
4046,Signal Data,5.0,2.0,0.0,0.0,1.0,2.0,0.0,8.0,1.0
4047,WEEL,4.0,3.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
4048,Alpaca,7.0,2.0,1.0,0.0,0.0,7.0,0.0,9.0,1.0


In [4]:
#print value_counts of each column of feature dataframe
for i in features.columns: 
    x= features[i].value_counts()
    print("column: ",i,", value is:",x)
    print()

column:  company_name , value is: Upside         3
Lendable       2
Vault          2
Moka           2
Float          2
              ..
Acre           1
SmartKassa     1
OPER           1
AND Global     1
Travala.com    1
Name: company_name, Length: 4031, dtype: int64

column:  num_funding_rounds , value is: 2.0     1311
3.0     1018
4.0      684
5.0      417
6.0      241
7.0      158
8.0       98
9.0       47
10.0      26
11.0      20
14.0       8
12.0       7
13.0       6
15.0       3
16.0       2
23.0       1
17.0       1
19.0       1
20.0       1
Name: num_funding_rounds, dtype: int64

column:  fd_rd_latest_investment , value is: 1.0    1741
2.0     900
3.0     533
0.0     495
4.0     242
5.0      88
6.0      29
7.0      22
Name: fd_rd_latest_investment, dtype: int64

column:  cat_lending_invests , value is: 0.0    3122
1.0     928
Name: cat_lending_invests, dtype: int64

column:  cat_payments , value is: 0.0    2789
1.0    1261
Name: cat_payments, dtype: int64

column:  cat_commerc

In [5]:
#print value_counts of each column of ylabel dataframe
for i in ylabel.columns: 
    x= ylabel[i].value_counts()
    print("column: ",i,", value is:",x)
    print()

column:  average_momentum , value is: -0.333333    5
 2.380480    3
 0.111111    2
 0.811275    2
 0.760427    2
            ..
-0.239214    1
 0.508306    1
 1.116198    1
-0.055383    1
 1.250000    1
Name: average_momentum, Length: 4015, dtype: int64



In [6]:
###### DO qcut for all ylabels ####### 

binary_labels = [0,1]

ylabel["average_momentum"]= pd.qcut(ylabel["average_momentum"], 2, labels = binary_labels)

ylabel

/Users/apple/opt/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,average_momentum
0,1
1,1
2,1
3,0
4,0
...,...
4045,1
4046,1
4047,1
4048,0


In [7]:
#Train model to predict each ylabel

X_train, X_test, y_train, y_test = train_test_split(features.drop(columns=["company_name"]), ylabel["average_momentum"], test_size=0.3, random_state=5)
model=DecisionTreeClassifier()
model.fit(X_train, y_train)
    
# use the trained model to predict test set for evaluation
pred_test = model.predict(X_test)

# print out evaluation result
print("Predicting: ", i)
    
print("Accuracy:{}".format(accuracy_score(y_test, pred_test, normalize=True, sample_weight=None)))
print("Classification Report:\n{}".format(classification_report(y_test, pred_test)))
print("Confusion Matrix:\n{}".format(confusion_matrix(y_test, pred_test)))
print("--------------------------------------------------------------------------")
prediction=model.predict(features.drop(columns=["company_name"]))
result= features
result["tree_prediction"]=prediction

Predicting:  average_momentum
Accuracy:0.5209876543209877
Classification Report:
              precision    recall  f1-score   support

           0       0.49      0.55      0.52       566
           1       0.56      0.50      0.53       649

    accuracy                           0.52      1215
   macro avg       0.52      0.52      0.52      1215
weighted avg       0.53      0.52      0.52      1215

Confusion Matrix:
[[311 255]
 [327 322]]
--------------------------------------------------------------------------


/Users/apple/opt/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [8]:
result["tree_prediction"].value_counts()

0    2229
1    1821
Name: tree_prediction, dtype: int64

In [9]:
result["average_momentum"]= df["average_momentum"]

/Users/apple/opt/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
result= result.sort_values(by="average_momentum", ascending=False)

In [11]:
result

,company_name,num_funding_rounds,fd_rd_latest_investment,cat_lending_invests,cat_payments,cat_commerce_shopping,first_fund_investor_count,first_fund_post_money,num_exec,fd_rd_num_invested_by_top_100,tree_prediction,average_momentum
1223,BrikkApp,2.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0,7.921425
3467,Beatdapp,4.0,1.0,0.0,0.0,0.0,10.0,0.0,8.0,1.0,1,7.920122
1118,Mobius,2.0,1.0,0.0,1.0,0.0,1.0,0.0,4.0,0.0,1,7.904976
1968,Blox,2.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,1,7.893792
1910,SONM,2.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0,7.833699
...,...,...,...,...,...,...,...,...,...,...,...,...
2173,Deep Labs,4.0,2.0,1.0,1.0,0.0,0.0,0.0,10.0,0.0,1,-7.395592
2689,CreditSuppliers,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,-7.668494
268,Swarm,2.0,1.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,0,-7.779349
3932,CarePay International,2.0,2.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1,-7.827990


In [12]:
counter= 0
output=[]

for i in range(result["company_name"].count()): 
    if counter== 400: 
        break
    if result.iloc[i]['tree_prediction']==1: 
        output.append(result.iloc[i]['company_name'])
        counter= counter+1
        
    
output

['Beatdapp',
 'Mobius',
 'Blox',
 'DealCircle',
 'Dispatch',
 'Finmark',
 'Handle',
 'Acudeen Technologies',
 'IDVigil',
 'Yotta Savings',
 '37coins',
 'Perch Credit',
 'Benepass',
 'Flex',
 'TiENPAY',
 'Abacus',
 'Avenews-GT',
 'CoinTracker',
 'Wunderite',
 'TrustToken',
 'Pipe',
 'Fangoo',
 'Startwise Inc',
 'Spacemesh',
 'Priceonomics',
 'CanYa',
 'BlockCypher',
 'OnePipe',
 'FreightBro',
 'Inkrypt',
 'Centrality',
 'BANKEX',
 'Zettabyte Pte Ltd',
 'Postmaster',
 'Numbers Protocol',
 'Grain',
 'Lending Loop',
 'Progcap',
 'ReadyForZero',
 'Moov',
 'SPARQ',
 'SmartKassa',
 'CoinLoan',
 'Qichacha',
 '1inch Exchange',
 'BlockEx',
 'Groundhog',
 'AND CO',
 'PATRON',
 'ArgonCredit',
 'Stem',
 'The Currency Account',
 'Coinbase CBT',
 'TenX',
 'Clara Lending',
 'TONIK',
 'Digital Claim',
 'Fuzhi Zhijia',
 'Notabene',
 'worig',
 'Paykii',
 "Let'sGrant",
 'StormX',
 'TrustaBit',
 'Veriff',
 'Venture For Canada',
 'CodeIsTrust',
 'Mineful',
 'Ankr',
 'SwissBorg',
 'Zilch Technology',
 'Prime

# Poly Regression (WILL DO IN 3rd iteration) 

In [13]:
# # Importing Linear Regression
# from sklearn.linear_model import LinearRegression
# # importing libraries for polynomial transform
# from sklearn.preprocessing import PolynomialFeatures

In [14]:
# feature= df[["first_fund_raised"]]
# ylabel=df[["average_momentum"]]
# ylabel

In [15]:
# ###### Normalize the result #######

# # define independent variables/attirbutes/features
# #x = features.drop(columns=['mean_norm_amount_delta', "company_uuid"])
# x= feature[["first_fund_raised"]]
# x=x[x["first_fund_raised"]!=0]
# x_max=x.max()
# x_min=x.min()
# x= (x-x_min)/(x_max-x_min)
# y = df[["first_fund_raised", "average_momentum"]]
# y=y[y["first_fund_raised"]!=0]
# y=y.drop(columns=["first_fund_raised"])

# #Change degree of x to fit into the model

# pre_process= PolynomialFeatures(degree=2)

# x_poly= pre_process.fit_transform(x)
# x_poly

# # Training the model
# pr_model= LinearRegression()

# # Fit our preprocessed data to the polynomial regression model
# pr_model.fit(x_poly, y)
# # Store our predicted ROI values in the variable y_pred
# y_pred = pr_model.predict(x_poly)

# # Plot our model on our data
# plt.scatter(x, y, c="orange")
# plt.xlabel(x.columns.values)
# plt.ylabel("ROI")
# plt.xlim([0,0.3])
# plt.plot(x, y_pred)

In [16]:
# num_of_top=400

# top_index= sorted(range(len(y_pred)), key=lambda i: y_pred[i], reverse=True)[:num_of_top]